# Genetic Algorithms Basics

Genetic Algorithms borrow inspiration from biological inheritance processes to attempt to solve optimization problems by generating a "population" of possible solutions (chromosomes) from which the most "fit" are selected for mating and mutation. By iterating this process over generations we expect the population to slowly converge upon the most "fit" region of our problem space which would correspond to the solution to the optimization task.

## Problem Representation (mapping)

In MGSurvE we are trying to solve the question **"Given a heterogeneous environment and a limited number of traps, where should we place the devices?"**, so this is the optimization task we need to map into a genetic algorithm paradigm. To do this, the positions of the traps are encoded into the chromosomes of the populations in pairs of alleles as follows:

![](../../media/fitness.png)

where a chromosome (or individual) would contain the position of the full set of traps. Once we have set the position of our traps we can calculate our fitness metric ($\phi$ in this case) by computing the time it would take for a random walker to fall into a trap given the current setup (more on this in the [following section](#fitness)). 

## Optimization Cycle

### Fitness

The fitness (or cost) function is the quantity that guides the artificial evolutionary process, as it is the factor we will try to maximize or minimize. MGSurvE computes a summary statistic on the time it would take for a random walker on our network to fall into an absorbing state (a trap) given that it started from any site in our landscape and then attempts to minimize it. Any function that operates on a list of elements can be used for this purpose, for example:

* Using the mean (`np.mean`) or total (`np.sum`) times until trapped usually results in a balanced placement of traps.
* Minimizing the maximum (`np.max`) time it would take for a mosquito to fall into an absorbing state results in the optimization giving more priority to more remote areas of the landscape.
* Other statistics such as the median (`np.median`) or even quantiles could potentially be used as long as they return a single quantity from a list of elements.

![](../../media/fits.png)

Calculating the fitness function is the most computationally-intensive part of the optimization process, as it involves solving [Markov's Fundamental Matrix](https://en.wikipedia.org/wiki/Absorbing_Markov_chain) on each potential solution of our landscape, and as such reducing the landscape and number of traps usually results in large speedups in the optimization process. 

For all the details and equations of how we compute our base fitness function have a look at our ["Mathematical Formulation"](../more/math.ipynb) document!

### Selection

The selection process involves choosing individuals from our pool in order for them to populate the next generation of our evolutionary process (usually in a scheme that favors the fittest individuals). MGSurvE is compatible with any of the [DEAP](https://deap.readthedocs.io/en/master/api/tools.html#operators-1) selection operators but uses the [tournament operator](https://deap.readthedocs.io/en/master/api/tools.html#deap.tools.selTournament) by default. In its most basic implementation, this operator randomly takes a defined number of tournament participants (tournament size) from the initial population, selects the fittest from them and repeats the process until we have the desired population size in our next generation.

![](../../media/selection.png)


### Crossover

This step involves taking pairs of chromosomes and "mixing" their alleles to simulate mating processes. MGSurvE provides extensions to [DEAP's](https://deap.readthedocs.io/en/master/api/tools.html#operators-1) [cxBlend](https://deap.readthedocs.io/en/master/api/tools.html#deap.tools.cxBlend) crossover operation for continuous optimization, along with [cxUniform](https://deap.readthedocs.io/en/master/api/tools.html#deap.tools.cxUniform) to be used both in continuous and discrete optimization processes. It's worth noting that these two variants are provided for cases in which immovable traps exist, if no fixed traps are present in the landscape any of the  [DEAP's](https://deap.readthedocs.io/en/master/api/tools.html#operators-1) crossover operators can be used.

![](../../media/crossover.png)

### Mutation

The final step in our evolutionary process is to allow the alleles of selected individuals in our population to "mutate" as to allow for new possible solutions to be generated. Our base MGSurvE implementation provides extensions to the [mutGaussian](https://deap.readthedocs.io/en/master/api/tools.html#deap.tools.mutGaussian) operator for continuous optimization operations along with a random uniform allele replacement one for discrete tasks. Same as with the crossover operation, these extensions are provided for cases in which immovable traps exist, if no fixed traps are present in our landscape we can use any of the available [DEAP](https://deap.readthedocs.io/en/master/api/tools.html#operators-1) mutation operations.

![](../../media/mutation.png)

<hr>

# More Information

* [Holland's original paper](https://link.springer.com/chapter/10.1007/978-1-4684-8941-5_21)
* [DEAP's documentation](https://deap.readthedocs.io/en/master/)
* [Fahmi Nurfikri's illustrated tutorial](https://towardsdatascience.com/an-illustrated-guide-to-genetic-algorithm-ec5615c9ebe)
* [MGSurvE's Webinar "Mathematical Formulation"](../more/math.ipynb)
* [MGSurvE's preprint](https://www.biorxiv.org/content/10.1101/2023.06.26.546301v1)